In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [13]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import requests
from io import BytesIO
from PIL import Image
import zipfile
import json

import tarfile
import numpy as np
from dataclasses import dataclass, field
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import pandas as pd
import yaml
import glob
import cv2

In [15]:
def merge_json_files(file_paths,output_file):
    merged_data = {"images": [], "annotations": [], "categories": []}

    # Iterate through each file
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            data = json.load(file)

            # Merge images
            merged_data["images"].extend(data["images"])

            # Merge annotations
            merged_data["annotations"].extend(data["annotations"])

            # Merge categories
            merged_data["categories"].extend(data["categories"])

    with open(output_file, 'w') as outfile:
        json.dump(merged_data, outfile,indent=4)


In [16]:
file_paths = ['./horse10_annotations/annotations/horse10-train-split1.json', './horse10_annotations/annotations/horse10-train-split2.json', './horse10_annotations/annotations/horse10-train-split3.json']  # Replace with the paths of your JSON files
output_file = 'horse10_annotations/annotations/horse10-train.json'  # Path of the output merged JSON file

merge_json_files(file_paths, output_file)

In [17]:
file_paths = ['./horse10_annotations/annotations/horse10-test-split1.json', './horse10_annotations/annotations/horse10-test-split2.json', './horse10_annotations/annotations/horse10-test-split3.json']  # Replace with the paths of your JSON files
output_file = './horse10_annotations/annotations/horse10-test.json'  # Path of the output merged JSON file

merge_json_files(file_paths, output_file)

In [18]:
def split_merged_json(input_file, train_output_file, valid_output_file, split_ratio=0.5):
    with open(input_file, 'r') as file:
        data = json.load(file)
    train_size = int(len(data['images']) * split_ratio)
    train_data = {key: data[key][:train_size] for key in data.keys()}
    valid_data = {key: data[key][train_size:] for key in data.keys()}
    with open(train_output_file, 'w') as train_file:
        json.dump(train_data, train_file, indent=4)
    with open(valid_output_file, 'w') as valid_file:
        json.dump(valid_data, valid_file, indent=4)

# Example usage:
input_file = './horse10_annotations/annotations/horse10-train.json'  # Replace with the path to your original JSON file
train_output_file = './horse10_annotations/annotations/train.json'  # Path for the training set JSON file
valid_output_file = './horse10_annotations/annotations/valid.json'  # Path for the validation set JSON file

split_merged_json(input_file, train_output_file, valid_output_file)

In [21]:
"""DATA_DIR = "/horse10_annotations/"

TRAIN_DIR         = f"train"
TRAIN_FOLDER_IMG    = f"images"
TRAIN_FOLDER_LABELS = f"labels/"

TRAIN_IMG_PATH   = os.path.join(DATA_DIR, TRAIN_DIR, TRAIN_FOLDER_IMG)
TRAIN_LABEL_PATH = os.path.join(DATA_DIR, TRAIN_DIR, TRAIN_FOLDER_LABELS)

print(TRAIN_LABEL_PATH)

VALID_DIR           = f"valid"
VALID_FOLDER_IMG    = f"images"
VALID_FOLDER_LABELS = f"labels"

VALID_IMG_PATH   = os.path.join(DATA_DIR, VALID_DIR, VALID_FOLDER_IMG)
VALID_LABEL_PATH = os.path.join(DATA_DIR, VALID_DIR, VALID_FOLDER_LABELS)

print(TRAIN_IMG_PATH)
print(TRAIN_LABEL_PATH)
print(VALID_IMG_PATH)
print(VALID_LABEL_PATH)


os.makedirs(TRAIN_IMG_PATH, exist_ok=True)
os.makedirs(TRAIN_LABEL_PATH, exist_ok=True)
os.makedirs(VALID_IMG_PATH, exist_ok=True)
os.makedirs(VALID_LABEL_PATH, exist_ok=True)"""

'DATA_DIR = "/horse10_annotations/"\n\nTRAIN_DIR         = f"train"\nTRAIN_FOLDER_IMG    = f"images"\nTRAIN_FOLDER_LABELS = f"labels/"\n\nTRAIN_IMG_PATH   = os.path.join(DATA_DIR, TRAIN_DIR, TRAIN_FOLDER_IMG)\nTRAIN_LABEL_PATH = os.path.join(DATA_DIR, TRAIN_DIR, TRAIN_FOLDER_LABELS)\n\nprint(TRAIN_LABEL_PATH)\n\nVALID_DIR           = f"valid"\nVALID_FOLDER_IMG    = f"images"\nVALID_FOLDER_LABELS = f"labels"\n\nVALID_IMG_PATH   = os.path.join(DATA_DIR, VALID_DIR, VALID_FOLDER_IMG)\nVALID_LABEL_PATH = os.path.join(DATA_DIR, VALID_DIR, VALID_FOLDER_LABELS)\n\nprint(TRAIN_IMG_PATH)\nprint(TRAIN_LABEL_PATH)\nprint(VALID_IMG_PATH)\nprint(VALID_LABEL_PATH)\n\n\nos.makedirs(TRAIN_IMG_PATH, exist_ok=True)\nos.makedirs(TRAIN_LABEL_PATH, exist_ok=True)\nos.makedirs(VALID_IMG_PATH, exist_ok=True)\nos.makedirs(VALID_LABEL_PATH, exist_ok=True)'

In [9]:
import os

# Définir le répertoire de base
DATA_DIR = "/horse10_annotations"  # Chemin relatif, ajusté pour éviter le début par un slash

# Définir les sous-dossiers
TRAIN_DIR = "train"
TRAIN_FOLDER_IMG = "images"
TRAIN_FOLDER_LABELS = "labels"

# Créer des chemins complets
TRAIN_IMG_PATH = os.path.join(DATA_DIR, TRAIN_DIR, TRAIN_FOLDER_IMG).replace("\\", "/")
TRAIN_LABEL_PATH = os.path.join(DATA_DIR, TRAIN_DIR, TRAIN_FOLDER_LABELS).replace("\\", "/")

print(TRAIN_LABEL_PATH)  # Doit afficher un chemin avec des barres obliques

# Répéter pour le répertoire de validation
VALID_DIR = "valid"
VALID_FOLDER_IMG = "images"
VALID_FOLDER_LABELS = "labels"

VALID_IMG_PATH = os.path.join(DATA_DIR, VALID_DIR, VALID_FOLDER_IMG).replace("\\", "/")
VALID_LABEL_PATH = os.path.join(DATA_DIR, VALID_DIR, VALID_FOLDER_LABELS).replace("\\", "/")

# Créer les répertoires si non existants
os.makedirs(TRAIN_IMG_PATH, exist_ok=True)
os.makedirs(TRAIN_LABEL_PATH, exist_ok=True)
os.makedirs(VALID_IMG_PATH, exist_ok=True)
os.makedirs(VALID_LABEL_PATH, exist_ok=True)


/horse10_annotations/train/labels


In [23]:
import os
import json
from shutil import copyfile
def copy_images_from_json(input_file, output_path):
    # Open the JSON file
    with open(input_file, 'r') as file:
        json_data = json.load(file)

    # Iterate through the 'images' section of the JSON data
    for data in json_data['images']:
        img_file = data["file_name"]
        #print(img_file)
        filename = str(data['id']) + '.png'
        # Copy the image file to the specified output path
        copyfile(os.path.join('./horse10/', img_file),
                 os.path.join(output_path, filename))

In [25]:
# Example usage:
input_file_path = './horse10_annotations/annotations/train.json'
output_directory_path = './horse10_annotations/train/images'
copy_images_from_json(input_file_path, output_directory_path)

In [26]:
# Example usage:
input_file_path1 = './horse10_annotations/annotations/valid.json'
output_directory_path1 = './horse10_annotations/valid/images'
copy_images_from_json(input_file_path1, output_directory_path1)

In [27]:
with open(input_file_path, 'r') as file:
        json_data = json.load(file)

In [28]:
for img_data,annotation_data in  zip(json_data['images'],json_data['annotations']):
  print(img_data.keys())
  print(annotation_data['keypoints'])
  break

dict_keys(['id', 'file_name', 'height', 'width'])
[4.8, 153.29999999999998, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [2]:
CLASS_ID = 0

In [3]:
def create_yolo_boxes_kpts(img_size, boxes, lm_kpts):

    IMG_W, IMG_H = img_size
    # Modify kpts with visibilities as 1s to 2s.


    # Normalizing factor for bboxes and kpts.
    res_box_array = np.array([IMG_W, IMG_H, IMG_W, IMG_H])
    res_lm_array = np.array([IMG_W, IMG_H])

    # Normalize landmarks in the range [0,1].
    norm_kps_per_img = lm_kpts.copy()
    l=len(norm_kps_per_img)
    i=1
    while i<=l:
      norm_kps_per_img[i-1]=norm_kps_per_img[i-1]/IMG_W
      norm_kps_per_img[i]=norm_kps_per_img[i]/IMG_H
      i=i+3

    # Normalize bboxes in the range [0,1].
    norm_bbox_per_img = boxes / res_box_array

    # Create bboxes coordinates to YOLO.
    # x_c, y_c = x_min + bbox_w/2. , y_min + bbox_h/2.
    yolo_boxes = norm_bbox_per_img.copy()
    yolo_boxes[:2] = norm_bbox_per_img[:2] + norm_bbox_per_img[2:]/2.

    return yolo_boxes, norm_kps_per_img

In [31]:
def create_yolo_txt_files(json_file, LABEL_PATH):
    with open(json_file, 'r') as file:
        json_data = json.load(file)

    for img_data,annotation_data in  zip(json_data['images'],json_data['annotations']):




        IMAGE_ID = img_data["id"]

        IMG_WIDTH, IMG_HEIGHT = img_data["width"], img_data["height"]

        landmark_kpts  = np.nan_to_num(np.array(annotation_data["keypoints"], dtype=np.float32))
        landmarks_bboxes = np.array(annotation_data["bbox"], dtype=np.float32)
        if len(landmark_kpts)!=66:
          print("false")
        bboxes_yolo, kpts_yolo = create_yolo_boxes_kpts(
                                            (IMG_WIDTH, IMG_HEIGHT),
                                            landmarks_bboxes,
                                            landmark_kpts)

        TXT_FILE = str(IMAGE_ID)+".txt"

        with open(os.path.join(LABEL_PATH, TXT_FILE), "w") as f:

            x_c_norm, y_c_norm, box_width_norm, box_height_norm = round(bboxes_yolo[0],5),\
                                                                  round(bboxes_yolo[1],5),\
                                                                  round(bboxes_yolo[2],5),\
                                                                  round(bboxes_yolo[3],5),\

            kps_flattend = [round(ele,5) for ele in kpts_yolo.flatten().tolist()]
            line = f"{CLASS_ID} {x_c_norm} {y_c_norm} {box_width_norm} {box_height_norm} "
            line+= " ".join(map(str, kps_flattend))
            f.write(line)

In [32]:
input_file_path = './horse10_annotations/annotations/train.json'
create_yolo_txt_files(input_file_path, TRAIN_LABEL_PATH)

In [33]:
input_file_path = './horse10_annotations/annotations/valid.json'
create_yolo_txt_files(input_file_path, VALID_LABEL_PATH)

In [4]:
@dataclass(frozen=True)
class TrainingConfig:
    DATASET_YAML:   str = "horse-keypoints.yaml"
    MODEL:          str = "yolov8m-pose.pt"
    EPOCHS:         int = 100
    KPT_SHAPE:    tuple = (22,3)
    PROJECT:        str = "Horse_Keypoints"
    NAME:           str = f"{MODEL.split('.')[0]}_{EPOCHS}_epochs"
    CLASSES_DICT:  dict = field(default_factory = lambda:{0 : "horse"})

In [5]:
@dataclass(frozen=True)
class DatasetConfig:
    BATCH_SIZE:    int   = 16
    CLOSE_MOSAIC:  int   = 10
    MOSAIC:        float = 0.4
    FLIP_LR:       float = 0.0

In [6]:
train_config = TrainingConfig()
data_config = DatasetConfig()

In [10]:
current_dir = os.getcwd()

data_dict = dict(
                path      = DATA_DIR,
                train     = os.path.join(TRAIN_DIR, TRAIN_FOLDER_IMG),
                val       = os.path.join(VALID_DIR, VALID_FOLDER_IMG),
                names     = train_config.CLASSES_DICT,
                kpt_shape = list(train_config.KPT_SHAPE),
               )

with open(train_config.DATASET_YAML, "w") as config_file:
    yaml.dump(data_dict, config_file)

In [11]:
pose_model = model = YOLO(train_config.MODEL)

pose_model.train(data    = train_config.DATASET_YAML,
            epochs       = train_config.EPOCHS,
            batch        = data_config.BATCH_SIZE,
            project      = train_config.PROJECT,
            name         = train_config.NAME,
            close_mosaic = data_config.CLOSE_MOSAIC,
            mosaic       = data_config.MOSAIC,
            fliplr       = data_config.FLIP_LR, 
            device = 0
           )

Ultralytics YOLOv8.2.2  Python-3.12.1 torch-2.2.2+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
engine\trainer: task=pose, mode=train, model=yolov8m-pose.pt, data=horse-keypoints.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=Horse_Keypoints, name=yolov8m-pose_100_epochs4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False,

train: Scanning D:\horse10_annotations\train\labels.cache... 2876 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2876/2876 [00:00<?, ?it/s]
val: Scanning D:\horse10_annotations\valid\labels.cache... 884 images, 0 backgrounds, 0 corrupt: 100%|██████████| 884/884 [00:00<?, ?it/s]


Plotting labels to Horse_Keypoints\yolov8m-pose_100_epochs4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 83 weight(decay=0.0), 93 weight(decay=0.0005), 92 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to Horse_Keypoints\yolov8m-pose_100_epochs4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      2.462       10.9     0.7134      4.902      2.942         18        640:   3%|▎         | 5/180 [01:56<1:08:03, 23.33s/it]


KeyboardInterrupt: 

In [ ]:
ckpt_path  = os.path.join(train_config.PROJECT, train_config.NAME, "weights", "best.pt")
model_pose = YOLO(ckpt_path)

metrics = model_pose.val()